<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/210121_Linear_Regression_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 210121 Linear Regression 2편

어제는 선형회귀의 가장 기본적인 증명과 어떻게 결과를 얻어내는지를 알았다면 오늘 볼 사항은 
- 범주형 변수가 있는 경우에는?
- 선형회귀모델에 가정은 없는가?
- 계수와 모델이 얼마나 의미있는가?

이 3가지에 대해 정리하려고 한다.

## 1. 확률론적 선형 회귀모형 및 선형회귀모델의 가정

앞에서 본 결과 모델을 일단 다시 가져와보자.
조건수 및 스케일링은 고려하지 않으려고 한다.

In [ ]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()

dfX0 = pd.DataFrame(boston.data, columns=boston.feature_names)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()
print(result_boston2.summary())

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Wed, 20 Jan 2021   Prob (F-statistic):          6.72e-135
Time:                        23:51:22   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.4595      5.103      7.144      0.0

의문이 하나 드는 것은, 계수가 저렇게 딱 결정되고 마는건가? 저 가중치마다 오차 범위가 있을 건데 실질적인 결론이 될 것인가? 이런 의문이 생길 수 있다.

즉, 우리는 **가중치의 오차범위**를 구해야 할 것이다. 하지만 어떻게?
이것에서 나온 생각이 바로 **확률론적 선형 회귀모형**이다.
이는 데이터가 확률 변수로 부터 생성된 표본이라고 가정한다.

하지만, 이를 위해서는 몇 가지 가설이 필요하다. 이 가설들을 정리하려고 한다.

- 오차는 정규분포를 띈다. **독립변수나 종속변수 자체가 정규분포일 필요는 없다.**

- 오차는 독립변수의 크기와 상관없이 기댓값이 0이다.
$$ E(\epsilon | x) = 0$$

- $i$번째 error와 $j$번째 error는 독립변수와 상관없이 독립이다. 즉,
$$ Cov[\epsilon_{i}, \epsilon_{j} | x] = 0$$

- $i$번째 error와 $j$번째 error는 분산이 동일하다.
$$ Cov(\epsilon) = \sigma^{2}I$$

## 2. 계수의 유의미성 검증

우리는 가중치 $w$의 예측치인 $\hat{w}$ 에 대하여 표준오차, 즉 공분산 행렬에서의 대각성분,
$$ Var[\hat{w_{i}}]$$ 을 고려한다.

더 정혹히는, 실제 가중치와 우리가 추정한 가중치 계수의 차이를 표준 오차로 나눈 값인
$$ \frac{\hat{w_{i}}-w_{i}}{se_{i}} $$
에 대해 고려할 것이다.

결론부터 말하자면, 표본 데이터의 수가 $N$,  $X$행렬의 열의 수가 $K$일때,

$$ \frac{\hat{w_{i}}-w_{i}}{se_{i}} \sim t_{N-K} $$

임을 얻어낼 수 있다. 이에 대한 증명은 추후 첨부하려 한다.

아무튼, 즉 각 계수마다 $t$분포 검정을 통하여 실제로 그 계수가 유의미한지를 얻어낼 수 있다.

이 때, 귀무가설은
$$H_{0}: w_{i}=0 $$

유의확률이 유의수준 이하일 경우, 귀무가설이 기각되어 $w_{i}$가 0일 가능성이 낮지만 그 반대일 경우 $w_{i}$가 0이 아니라고 할 수 없다.



In [ ]:
print(result_boston2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Thu, 21 Jan 2021   Prob (F-statistic):          6.72e-135
Time:                        00:36:45   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.4595      5.103      7.144      0.0

이를 바탕으로 다시 boston 문제로 돌아와보면,
CHAS와 AGE변수는 제하는 것이 합리적임을 알 수 있다.

## 3. 범주형 변수의 처리
범주형 변수도 회귀분석에 필요한 경우가 있을 법 한데, 이에 대해서는 당연히 전처리를 해줘야 할 것이다. 이 때 쓰이는 것이 바로 **One-hot Encoding.** 

즉, 더미 변수를 만들어야 할 것이다.

예를들어, 하나의 특성이 남성/여성 으로 나뉘어있는데 남성이 0, 여성이 1 이렇게 데이터상에 적혀 있다고 생각해보자.

이걸 그대로 회귀분석에 넣는다면 0과 1을 있는 그대로 읽을 것이기에 문제가 될 것이다.

따라서 남성을 (1, 0),  여성을 (0, 1)로 코딩한다면
문제가 없을 것이다.

조금 더 구체적으로 어떤식으로 식으로 표현하는지 본다면, 범주값이 2가지라고 가정시

$$ x_{1} = A \rightarrow d_{1A}=1, d_{1B}=0 $$
$$ x_{1} = B \rightarrow d_{1A}=0, d_{1B}=1 $$

이 된다.

다시 말해,

$$ x_{1} = A \rightarrow \hat{y} = w_{1A}+w{2}x{2}+...+w_{n}x{n} $$
$$ x_{1} = B \rightarrow \hat{y} = w_{1B}+w{2}x{2}+...+w_{n}x{n} $$

즉, 이런 식으로 표기되기에 범주값을 제외하고는 다른 상수항이 있어서는 안 된다.